In [2]:
# Optimizations for HP Mini
import cv2
import json
import numpy as np
import pandas as pd
from pathlib import Path
import os
from ultralytics import YOLO
import gc
import time

# Optimized configurations for HP Elite Mini 800 G9 (i5-12500T) with 16GB RAM
ELITE_MINI_CONFIG = {
    'yolo_model': 'yolov8s.pt',  # Small model (good balance)
    'img_size': 640,  # Standard size
    'batch_size': 6,  # Increased batch size for 16GB RAM
    'epochs': 30,  # More epochs with sufficient RAM
    'patience': 7,  # Longer patience
    'target_size': (224, 224),  # Standard target size
    'frame_skip': 3,  # Process more frames
    'max_frames': 8000,  # Handle more frames with 16GB
    'workers': 8,  # More workers for faster data loading
    'cache_ram': True,  # Enable RAM caching
}

class OptimizedDataGenerator:
    """Memory-efficient data generator for HP Mini"""
    
    def __init__(self, data_info, config=ELITE_MINI_CONFIG):  # Fixed: Changed HP_MINI_CONFIG to ELITE_MINI_CONFIG
        self.data_info = data_info[:config['max_frames']]  # Limit data
        self.config = config
        self.current_idx = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current_idx >= len(self.data_info):
            raise StopIteration
            
        data = self.data_info.iloc[self.current_idx]
        self.current_idx += 1
        
        try:
            # Load and process frame
            cap = cv2.VideoCapture(str(data['video_path']))
            cap.set(cv2.CAP_PROP_POS_FRAMES, data['frame_num'])
            ret, frame = cap.read()
            cap.release()
            
            if ret:
                # Resize to smaller size
                frame = cv2.resize(frame, self.config['target_size'])
                frame = frame.astype(np.float32) / 255.0
                return frame, data['behavior_encoded']
            else:
                return self.__next__()  # Skip failed frames
                
        except Exception as e:
            print(f"Error loading frame: {e}")
            return self.__next__()

class EliteMiniYOLODetector:
    """Optimized YOLO detector for HP Elite Mini 800 G9"""
    
    def __init__(self, config=ELITE_MINI_CONFIG):
        self.config = config
        self.model = YOLO(config['yolo_model'])
        
    def train_optimized(self, data_yaml):
        """Train with Elite Mini optimizations"""
        print("Training with Elite Mini 800 G9 optimizations (16GB RAM)...")
        
        results = self.model.train(
            data=data_yaml,
            epochs=self.config['epochs'],
            imgsz=self.config['img_size'],
            batch=self.config['batch_size'],
            device='cpu',
            workers=self.config['workers'],
            patience=self.config['patience'],
            cache=self.config['cache_ram'],  # Enable RAM caching
            save_period=10,
            verbose=True,
            amp=False,  # Keep disabled for CPU
            optimizer='AdamW',
            lr0=0.01,
            weight_decay=0.0005,
            momentum=0.937,
        )
        return results
    
    def detect_optimized(self, frame, conf=0.3):
        """Optimized detection for HP Mini"""
        # Resize frame to smaller size
        small_frame = cv2.resize(frame, (self.config['img_size'], self.config['img_size']))
        
        # Run inference
        results = self.model.predict(
            small_frame,
            conf=conf,
            device='cpu',
            verbose=False,
            half=False,  # Disable half precision
        )
        
        # Scale detections back to original size
        h, w = frame.shape[:2]
        scale_x = w / self.config['img_size']
        scale_y = h / self.config['img_size']
        
        detections = []
        for result in results:
            if result.boxes is not None:
                boxes = result.boxes.xyxy.cpu().numpy()
                confs = result.boxes.conf.cpu().numpy()
                
                for box, conf in zip(boxes, confs):
                    # Scale back to original size
                    x1, y1, x2, y2 = box
                    x1, x2 = int(x1 * scale_x), int(x2 * scale_x)
                    y1, y2 = int(y1 * scale_y), int(y2 * scale_y)
                    
                    detections.append({
                        'bbox': [x1, y1, x2-x1, y2-y1],
                        'confidence': float(conf)
                    })
        
        return detections

def prepare_standard_dataset(data_info, output_dir, config=ELITE_MINI_CONFIG):
    """Prepare standard dataset for Elite Mini"""
    output_dir = Path(output_dir)
    (output_dir / 'images').mkdir(parents=True, exist_ok=True)
    (output_dir / 'labels').mkdir(parents=True, exist_ok=True)
    
    # Can handle more frames
    limited_data = data_info.head(config['max_frames'])
    
    # Create data.yaml
    data_yaml = {
        'path': str(output_dir.absolute()),
        'train': 'images',
        'val': 'images',
        'names': ['pig'],
        'nc': 1
    }
    
    import yaml
    with open(output_dir / 'data.yaml', 'w') as f:
        yaml.dump(data_yaml, f)
    
    print(f"Processing {len(limited_data)} frames...")
    
    for idx, row in limited_data.iterrows():
        if idx % 200 == 0:
            print(f"Processed {idx}/{len(limited_data)} frames")
            gc.collect()
        
        try:
            # Load frame
            cap = cv2.VideoCapture(str(row['video_path']))
            cap.set(cv2.CAP_PROP_POS_FRAMES, row['frame_num'])
            ret, frame = cap.read()
            cap.release()
            
            if not ret:
                continue
            
            # Keep original size for better quality
            frame = cv2.resize(frame, (config['img_size'], config['img_size']))
            
            # Save image with good quality
            img_path = output_dir / 'images' / f"{row['video_id']}_{row['frame_num']}.jpg"
            cv2.imwrite(str(img_path), frame, [cv2.IMWRITE_JPEG_QUALITY, 95])
            
            # Process bbox (same as before)
            bbox = row['bbox']
            if isinstance(bbox, str):
                import ast
                bbox = ast.literal_eval(bbox)
            if isinstance(bbox, dict):
                x = bbox.get('x', 0)
                y = bbox.get('y', 0)
                w = bbox.get('width', bbox.get('w', 100))
                h = bbox.get('height', bbox.get('h', 100))
                bbox = [x, y, w, h]
            
            # Convert to YOLO format
            img_h, img_w = config['img_size'], config['img_size']
            x_center = (bbox[0] + bbox[2]/2) / img_w
            y_center = (bbox[1] + bbox[3]/2) / img_h
            width = bbox[2] / img_w
            height = bbox[3] / img_h
            
            # Clamp values
            x_center = max(0, min(1, x_center))
            y_center = max(0, min(1, y_center))
            width = max(0, min(1, width))
            height = max(0, min(1, height))
            
            # Save label
            label_path = output_dir / 'labels' / f"{row['video_id']}_{row['frame_num']}.txt"
            with open(label_path, 'w') as f:
                f.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
                
        except Exception as e:
            print(f"Error processing frame {idx}: {e}")
            continue

def load_annotated_data_info(data_root="C:/Users/2955352g/Desktop/pig_data_edinburgh"):
    """Load annotated data from various possible formats"""
    data_root = Path(data_root)
    
    print(f"Searching for data in: {data_root}")
    
    # Check what files exist in the data directory
    if not data_root.exists():
        print(f"Data directory {data_root} does not exist!")
        return pd.DataFrame()
    
    # List all files in the directory
    print("Files found in data directory:")
    for item in data_root.rglob("*"):
        if item.is_file():
            print(f"  {item.name} ({item.suffix})")
    
    # Try to load from common annotation formats
    annotation_data = []
    
    # Method 1: Try to load from CSV files
    csv_files = list(data_root.glob("*.csv"))
    if csv_files:
        print(f"\nFound {len(csv_files)} CSV files:")
        for csv_file in csv_files:
            print(f"  {csv_file.name}")
            try:
                df = pd.read_csv(csv_file)
                print(f"    Columns: {list(df.columns)}")
                print(f"    Shape: {df.shape}")
                annotation_data.append(df)
            except Exception as e:
                print(f"    Error reading {csv_file}: {e}")
    
    # Method 2: Try to load from JSON files
    json_files = list(data_root.glob("*.json"))
    if json_files:
        print(f"\nFound {len(json_files)} JSON files:")
        for json_file in json_files:
            print(f"  {json_file.name}")
            try:
                with open(json_file, 'r') as f:
                    data = json.load(f)
                print(f"    Keys: {list(data.keys()) if isinstance(data, dict) else 'List format'}")
                # Convert JSON to DataFrame format if needed
                if isinstance(data, list):
                    df = pd.DataFrame(data)
                    annotation_data.append(df)
                elif isinstance(data, dict):
                    df = pd.DataFrame([data])
                    annotation_data.append(df)
            except Exception as e:
                print(f"    Error reading {json_file}: {e}")
    
    # Method 3: Look for video files and create basic structure
    video_extensions = ['.mp4', '.avi', '.mov', '.mkv']
    video_files = []
    for ext in video_extensions:
        video_files.extend(data_root.glob(f"*{ext}"))
        video_files.extend(data_root.glob(f"**/*{ext}"))
    
    if video_files:
        print(f"\nFound {len(video_files)} video files:")
        for video_file in video_files[:5]:  # Show first 5
            print(f"  {video_file.name}")
        if len(video_files) > 5:
            print(f"  ... and {len(video_files) - 5} more")
    
    # If we found annotation data, try to standardize it
    if annotation_data:
        print(f"\nProcessing {len(annotation_data)} annotation files...")
        
        # Combine all annotation data
        combined_df = pd.concat(annotation_data, ignore_index=True)
        
        # Try to standardize column names
        standardized_df = standardize_annotation_format(combined_df, data_root)
        
        if not standardized_df.empty:
            print(f"Successfully loaded {len(standardized_df)} annotations")
            return standardized_df
    
    # If no annotation files found, create a basic structure from video files
    if video_files:
        print("\nNo annotation files found. Creating basic structure from video files...")
        return create_basic_annotations_from_videos(video_files)
    
    print("\nNo suitable data found!")
    return pd.DataFrame()

def standardize_annotation_format(df, data_root):
    """Try to standardize annotation format to expected columns"""
    print("Attempting to standardize annotation format...")
    print(f"Input columns: {list(df.columns)}")
    
    # Common column name mappings
    column_mappings = {
        # Video path variations
        'video_path': ['video_path', 'video_file', 'video', 'filename', 'file_path'],
        'video_id': ['video_id', 'video_name', 'id', 'video', 'file_id'],
        
        # Frame number variations
        'frame_num': ['frame_num', 'frame_number', 'frame', 'frame_id', 'timestamp'],
        
        # Bounding box variations
        'bbox': ['bbox', 'bounding_box', 'box', 'coordinates', 'bounds'],
        'x': ['x', 'x1', 'left', 'bbox_x'],
        'y': ['y', 'y1', 'top', 'bbox_y'],
        'width': ['width', 'w', 'bbox_width', 'bbox_w'],
        'height': ['height', 'h', 'bbox_height', 'bbox_h'],
        
        # Behavior/label variations
        'behavior': ['behavior', 'label', 'class', 'category', 'behavior_label'],
        'behavior_encoded': ['behavior_encoded', 'label_encoded', 'class_id'],
    }
    
    # Create new standardized dataframe
    standardized_data = []
    
    for idx, row in df.iterrows():
        try:
            # Extract video information
            video_path = None
            video_id = None
            
            for col in df.columns:
                if any(vid_col in col.lower() for vid_col in column_mappings['video_path']):
                    video_path = row[col]
                    break
            
            for col in df.columns:
                if any(vid_col in col.lower() for vid_col in column_mappings['video_id']):
                    video_id = row[col]
                    break
            
            # If video_path is relative, make it absolute
            if video_path and not os.path.isabs(video_path):
                video_path = data_root / video_path
            
            # Extract frame number
            frame_num = 0
            for col in df.columns:
                if any(frame_col in col.lower() for frame_col in column_mappings['frame_num']):
                    frame_num = row[col]
                    break
            
            # Extract bounding box
            bbox = None
            
            # Try to find bbox as single column
            for col in df.columns:
                if any(bbox_col in col.lower() for bbox_col in column_mappings['bbox']):
                    bbox = row[col]
                    break
            
            # If no single bbox column, try to construct from x,y,w,h
            if bbox is None:
                x, y, w, h = None, None, None, None
                
                for col in df.columns:
                    col_lower = col.lower()
                    if any(x_col in col_lower for x_col in column_mappings['x']):
                        x = row[col]
                    elif any(y_col in col_lower for y_col in column_mappings['y']):
                        y = row[col]
                    elif any(w_col in col_lower for w_col in column_mappings['width']):
                        w = row[col]
                    elif any(h_col in col_lower for h_col in column_mappings['height']):
                        h = row[col]
                
                if all(v is not None for v in [x, y, w, h]):
                    bbox = [x, y, w, h]
            
            # Extract behavior
            behavior = None
            behavior_encoded = 0
            
            for col in df.columns:
                if any(beh_col in col.lower() for beh_col in column_mappings['behavior']):
                    behavior = row[col]
                    break
            
            for col in df.columns:
                if any(beh_col in col.lower() for beh_col in column_mappings['behavior_encoded']):
                    behavior_encoded = row[col]
                    break
            
            # Create standardized entry
            if video_path and bbox:
                standardized_data.append({
                    'video_path': str(video_path),
                    'video_id': video_id or f"video_{idx}",
                    'frame_num': int(frame_num),
                    'bbox': bbox,
                    'behavior': behavior or "pig_behavior",
                    'behavior_encoded': int(behavior_encoded)
                })
                
        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            continue
    
    if standardized_data:
        result_df = pd.DataFrame(standardized_data)
        print(f"Successfully standardized {len(result_df)} annotations")
        return result_df
    else:
        print("Could not standardize any annotations")
        return pd.DataFrame()

def create_basic_annotations_from_videos(video_files):
    """Create basic annotations from video files (for testing)"""
    print("Creating basic annotations from video files...")
    
    annotations = []
    
    for video_file in video_files[:10]:  # Limit to first 10 videos
        try:
            # Get video info
            cap = cv2.VideoCapture(str(video_file))
            if not cap.isOpened():
                continue
                
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            cap.release()
            
            # Create annotations for every 30th frame (sample)
            for frame_num in range(0, min(total_frames, 3000), 30):
                annotations.append({
                    'video_path': str(video_file),
                    'video_id': video_file.stem,
                    'frame_num': frame_num,
                    'bbox': [100, 100, 200, 200],  # Dummy bbox - you'll need real annotations
                    'behavior': 'pig_behavior',
                    'behavior_encoded': 0
                })
        except Exception as e:
            print(f"Error processing video {video_file}: {e}")
            continue
    
    if annotations:
        print(f"Created {len(annotations)} basic annotations")
        print("⚠️  WARNING: Using dummy bounding boxes! You need real annotations for proper training.")
        return pd.DataFrame(annotations)
    else:
        return pd.DataFrame()

def elite_mini_pipeline(data_root):
    """Optimized pipeline for HP Elite Mini 800 G9"""
    print("Starting Elite Mini 800 G9 optimized pipeline...")
    
    # Load data
    print("Loading data information...")
    data_info = load_annotated_data_info()
    
    if data_info.empty:
        print("No data found - please implement load_annotated_data_info()")
        return None
    
    # Can handle more data
    data_info = data_info.head(ELITE_MINI_CONFIG['max_frames'])
    print(f"Using {len(data_info)} frames for training")
    
    # Prepare dataset
    print("Preparing dataset...")
    prepare_standard_dataset(data_info, 'elite_mini_dataset')
    
    # Initialize detector
    detector = EliteMiniYOLODetector()
    
    # Train with optimizations
    print("Training detector (estimated 2-4 hours)...")
    start_time = time.time()
    
    results = detector.train_optimized('elite_mini_dataset/data.yaml')
    
    training_time = time.time() - start_time
    print(f"Training completed in {training_time/3600:.2f} hours")
    
    return detector

def test_elite_mini_video(detector, video_path):
    """Test video processing on Elite Mini"""
    if not os.path.exists(video_path):
        print(f"Video file {video_path} does not exist")
        return
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Cannot open video {video_path}")
        return
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Processing video with {total_frames} frames")
    
    frame_count = 0
    processed_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        
        # Process every 5th frame (reasonable for 12-thread CPU)
        if frame_count % ELITE_MINI_CONFIG['frame_skip'] == 0:
            start_time = time.time()
            
            detections = detector.detect_optimized(frame)
            
            process_time = time.time() - start_time
            processed_count += 1
            
            print(f"Frame {frame_count}: {len(detections)} detections, "
                  f"Processing time: {process_time:.2f}s")
            
            # Less frequent memory cleanup
            if processed_count % 100 == 0:
                gc.collect()
    
    cap.release()
    print(f"Processed {processed_count} frames out of {total_frames}")
    print(f"Average processing speed: {processed_count/total_frames*100:.1f}% of frames")

# Enhanced memory monitoring for 16GB system
def monitor_memory():
    """Monitor detailed memory usage"""
    import psutil
    
    # Overall system memory
    memory = psutil.virtual_memory()
    print(f"Total RAM: {memory.total / 1024**3:.1f} GB")
    print(f"Available RAM: {memory.available / 1024**3:.1f} GB")
    print(f"Used RAM: {memory.used / 1024**3:.1f} GB ({memory.percent:.1f}%)")
    
    # Process-specific memory
    process = psutil.Process()
    process_memory = process.memory_info()
    print(f"Process RSS: {process_memory.rss / 1024**3:.2f} GB")
    print(f"Process VMS: {process_memory.vms / 1024**3:.2f} GB")
    
    # Check if we're approaching memory limits
    if memory.percent > 85:
        print("⚠️  WARNING: High memory usage detected!")
        print("Consider reducing batch_size or max_frames")
    elif memory.percent > 70:
        print("ℹ️  Memory usage is getting high, monitoring recommended")
    else:
        print("✅ Memory usage is healthy")

def optimize_for_16gb():
    """Suggest optimizations based on current memory usage"""
    import psutil
    memory = psutil.virtual_memory()
    
    if memory.available < 4 * 1024**3:  # Less than 4GB available
        print("🔧 Reducing batch size for low available memory")
        ELITE_MINI_CONFIG['batch_size'] = 2
        ELITE_MINI_CONFIG['max_frames'] = 4000
        ELITE_MINI_CONFIG['workers'] = 4
    elif memory.available < 6 * 1024**3:  # Less than 6GB available
        print("🔧 Using moderate settings for medium available memory")
        ELITE_MINI_CONFIG['batch_size'] = 4
        ELITE_MINI_CONFIG['max_frames'] = 6000
        ELITE_MINI_CONFIG['workers'] = 6
    else:
        print("✅ Using full settings for high available memory")
        # Use default settings from config

if __name__ == "__main__":
    # Monitor initial memory and optimize settings
    print("=== Initial System Status ===")
    monitor_memory()
    optimize_for_16gb()
    
    print("\n=== Starting Training Pipeline ===")
    
    # Run Elite Mini optimized pipeline
    detector = elite_mini_pipeline("C:/Users/2955352g/Desktop/pig_data_edinburgh")
    
    if detector:
        print("Training completed successfully!")
        
        # Monitor memory after training
        print("\n=== Post-Training Memory Status ===")
        monitor_memory()
        
        # Test on video
        test_video = "test_video.mp4"  # Adjust path
        if os.path.exists(test_video):
            test_elite_mini_video(detector, test_video)
    
    # Final memory check
    print("\n=== Final Memory Status ===")
    monitor_memory()

=== Initial System Status ===
Total RAM: 15.7 GB
Available RAM: 3.7 GB
Used RAM: 12.0 GB (76.6%)
Process RSS: 0.26 GB
Process VMS: 0.60 GB
ℹ️  Memory usage is getting high, monitoring recommended
🔧 Reducing batch size for low available memory

=== Starting Training Pipeline ===
Starting Elite Mini 800 G9 optimized pipeline...
Loading data information...
Searching for data in: C:\Users\2955352g\Desktop\pig_data_edinburgh
Files found in data directory:
  test_video.mp4 (.mp4)
  background.png (.png)
  background_depth.png (.png)
  color.mp4 (.mp4)
  depth.mp4 (.mp4)
  depth_scale.npy (.npy)
  inverse_intrinsic.npy (.npy)
  mask.png (.png)
  output.json (.json)
  rot.npy (.npy)
  times.txt (.txt)
  background.png (.png)
  background_depth.png (.png)
  color.mp4 (.mp4)
  depth.mp4 (.mp4)
  depth_scale.npy (.npy)
  inverse_intrinsic.npy (.npy)
  mask.png (.png)
  output.json (.json)
  rot.npy (.npy)
  times.txt (.txt)
  background.png (.png)
  background_depth.png (.png)
  color.mp4 (.mp4)


100%|██████████| 21.5M/21.5M [00:29<00:00, 755kB/s] 


Training detector (estimated 2-4 hours)...
Training with Elite Mini 800 G9 optimizations (16GB RAM)...
Ultralytics 8.3.166  Python-3.11.13 torch-2.5.1 CPU (12th Gen Intel Core(TM) i5-12500T)
engine\trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=elite_mini_dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=

100%|██████████| 755k/755k [00:00<00:00, 5.18MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           


...

      27/30         0G      1.154      1.369      1.423          2        640: 100%|██████████| 90/90 [01:20<00:00,  1.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 45/45 [00:28<00:00,  1.57it/s]

                   all        180        180      0.637      0.772      0.673      0.427
EarlyStopping: Training stopped early as no improvement observed in last 7 epochs. Best results observed at epoch 20, best model saved as best.pt.
To update EarlyStopping(patience=7) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



27 epochs completed in 0.905 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train\weights\best.pt, 22.5MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.166  Python-3.11.13 torch-2.5.1 CPU (12th Gen Intel Core(TM) i5-12500T)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 45/45 [00:27<00:00,  1.65it/s]


                   all        180        180      0.941      0.633      0.817      0.578
Speed: 0.7ms preprocess, 146.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\train
Training completed in 0.92 hours
Training completed successfully!

=== Post-Training Memory Status ===
Total RAM: 15.7 GB
Available RAM: 4.4 GB
Used RAM: 11.2 GB (71.7%)
Process RSS: 2.01 GB
Process VMS: 2.75 GB
ℹ️  Memory usage is getting high, monitoring recommended

=== Final Memory Status ===
Total RAM: 15.7 GB
Available RAM: 4.4 GB
Used RAM: 11.2 GB (71.7%)
Process RSS: 2.01 GB
Process VMS: 2.75 GB
ℹ️  Memory usage is getting high, monitoring recommended


kj
